In [ ]:
import os
import pandas as pd

for l in os.listdir("dataset/fermi_excel/Fermate"):
    df = pd.read_excel(f"dataset/fermi_excel/Fermate/{l}")
    name = l.replace(".xls", "")

    df.to_csv(f"dataset/fermi/Fermate/{name}.csv", index=False)

df = pd.read_excel(f"dataset/fermi_excel/FERMATE 2211 ACR.xls")
df.to_csv(f"dataset/fermi/FERMATE 2211 ACR.csv", index=False)

Now I iterate for every machine to get the data from the 3 datasets and merge them into one dataset, removing the ones that are not useful for the analysis.

IT1 ~ Now we can check for all the dataset and see what we can do with them

In [ ]:
import pandas as pd
from scripts.getDataset import getEntireDataset, forEveryMachine, cleanDataset

DEBUG = True
pd.set_option("display.max_rows", None)
pd.options.mode.copy_on_write = False

most_interesting = []

def fn(machineId, year, month):
    dataset = getEntireDataset(machineId, year, month, False)

    if dataset.empty:
        print("Skipping as empty", machineId, year, month)
        return

    # if dataset["EnergyConsumption"].dropna().eq(0).all():
    #     if DEBUG:
    #         print("Skipping as EnergyConsumption is all 0")
    #     return
    # if dataset["Fermate"].dropna().eq(0).all():
    #     if DEBUG:
    #         print("Skipping as Fermate is all 0")
    #     return

    # EC_na = dataset["EnergyConsumption"].isna().sum()
    # EC_len = dataset["EnergyConsumption"].shape[0]

    # ST_na = dataset["Fermate"].isna().sum()
    # ST_len = dataset["Fermate"].shape[0]

    # energyRate = EC_na/ EC_len
    # stopsRate = ST_na/ ST_len

    # if energyRate > 0.5 or stopsRate > 0.5:
    #     if DEBUG:
    #         print("Skipping as too many NaN values")
    #         print("\tEnergy Rate:", energyRate)
    #         print("\tStops Rate:", stopsRate)
    #     return

    dataset = cleanDataset(dataset)
    dataset.to_csv(
        f"dataset/results/it1/id-{machineId}_{year}-{month}.csv", index=False
    )

    most_interesting.append((dataset, machineId, year, month))

forEveryMachine(fn)

print("\n\n\nThe most interesting are the followings:")
for dataset, machineId, year, month in most_interesting:
    print(f"- Machine {machineId} - {year}/{month}")

# for dataset, machineId, year, month in most_interesting:
#     plot(dataset, machineId, year, month)

IT2 ~ It took 20 minutes to compute everything but now I want to look at the lifetime of the machines and see if I can find something interesting, merging the data into one file for machine

In [ ]:
from scripts.getSingleDataset.utils import getCleanDataset
import pandas as pd
import os


compete_dataset = {}

for path in os.listdir("dataset/results/it1"):
    # dataset.to_csv(f"dataset/results/it1/id-{machineId}_{year}-{month}.csv")

    splitted_filename = [p.split("-") for p in path.replace(".csv", "").split("_")]

    machineId = int(splitted_filename[0][1])
    year = int(splitted_filename[1][0])
    month = int(splitted_filename[1][1])

    d = getCleanDataset(f"dataset/results/it1/id-{machineId}_{year}-{month}.csv")

    # plot(d)
    value = [d]

    if machineId in compete_dataset:
        old_data = compete_dataset.get(machineId)

        for old in old_data:
            value.append(old)

    compete_dataset.update({machineId: value})

for machineId in compete_dataset.keys():
    d = pd.concat(compete_dataset.get(machineId))

    d.to_csv(f"dataset/results/it2/id-{machineId}.csv", mode="w", index=False)

IT3 ~ Now I use the pearson correlation to see the correlation between datasets

In [ ]:
import os
from scripts.getSingleDataset.utils import getCleanDataset
from scipy import stats
from scripts.plots import plot

files = [p for p in os.listdir("dataset/results/it2") if p.endswith(".csv")]

STR = "\t\t{:0.2f}\n\t\tP-Value:{:0.2f}"

def is_correlation_usable(data):
    computing_data = data.dropna()

    # Correlazione tra Energia e Produzioni
    energy_productions_corr, energy_productions_pvalue = stats.pearsonr(
        computing_data["EnergyConsumption"], computing_data["Productions"]
    )
    print("\tCorrelation between Energy Consumption and Productions")
    print(STR.format(energy_productions_corr, energy_productions_pvalue))
    if energy_productions_pvalue <= 0.05:
        print("\t\tSignificant")

    # Correlazione tra Energia e Fermate
    # energy_stops_corr, energy_stops_pvalue = stats.pearsonr(
    #     computing_data["EnergyConsumption"], computing_data["Fermate"]
    # )
    # print("\tCorrelation between Energy Consumption and Stops")
    # print(STR.format(energy_stops_corr, energy_stops_pvalue))
    # if energy_stops_pvalue <= 0.05:
    #     print("\t\tSignificant")

    # Correlazione tra Produzioni e Fermate
    # productions_stops_corr, productions_stops_pvalue = stats.pearsonr(
    #     computing_data["Productions"], computing_data["Stops"]
    # )
    # print("\tCorrelation between Productions and Stops")
    # print(STR.format(productions_stops_corr, productions_stops_pvalue))
    # if productions_stops_pvalue <= 0.05:
    #     print("\t\tSignificant")

    return True
    return (
        energy_productions_pvalue <= 0.05
        # or energy_stops_pvalue <= 0.05
        # or productions_stops_pvalue <= 0.05
    )

dfs = [(int(file.replace(".csv", "").split("-")[1]), getCleanDataset(f"dataset/results/it2/{file}")) for file in files]

usable_datasets = [df for df in dfs if is_correlation_usable(df[1])]

print("Usable datasets: ", len(usable_datasets))

tot = 0
for machineId, d in usable_datasets:
    d.to_csv(f"dataset/results/it3/id-{machineId}.csv", mode="w", index=False)
    tot += d.shape[0]
    print("- Machine", machineId)
    plot(d)

# print("Total:", tot)

Now I want to use a linear model to try to predict the lifetime of the machines

Now I try to add material to the model

In [12]:
import os
import pandas as pd
from scripts.getSingleDataset.getProductions import getProductionWithFixedComma
from sklearn.ensemble import HistGradientBoostingClassifier
pd.set_option("display.max_rows", None)

SHIFT_VALUE = 1
COLUMNS_TO_SHIFT = {
    "Productions": "Previous_Prod",
    "EnergyConsumption": "Previous_EC",
    "Material_HASH": "Previous_Material",
    "COD_ART_HASH": "Previous_COD_ART",
}

COLUMNS = [
    "MachineId",
    "Productions",
    "EnergyConsumption",
    "COD_ART_HASH",
    "Material_HASH",
]
COLUMNS += list(COLUMNS_TO_SHIFT.values())

# get materials
materials = getProductionWithFixedComma(f"dataset/fermi/FERMATE 2211 ACR.csv")[
    ["CODART", "ACR"]
].drop_duplicates()
materials.rename(columns={"CODART": "COD_ART", "ACR": "Material"}, inplace=True)

# get dataset
train_data = pd.DataFrame()
for file in os.listdir("dataset/results/it3"):
    data = pd.read_csv(f"dataset/results/it3/{file}")

    data.sort_values(by="START_DATE", inplace=True)

    data = pd.merge(data, materials, how="left")
    data["Material_HASH"] = data["Material"].apply(hash)

    data["COD_ART_HASH"] = data["COD_ART"].apply(hash)

    previous_size = data.shape[0]
    # add previous data
    for c in COLUMNS_TO_SHIFT.keys():
        data[COLUMNS_TO_SHIFT[c]] = data[c].shift(-SHIFT_VALUE)
    #     print(data[["START_DATE", "END_DATE", COLUMNS_TO_SHIFT[c], c]])
    assert previous_size == data.shape[0]

    data["MachineId"] = int(file.replace(".csv", "").split("-")[1])

    # data = data[data["Stop"] != "No Stop"]
    data.dropna()

    train_data = pd.concat([train_data, data])

def calculateModel(dataset: pd.DataFrame, model: HistGradientBoostingClassifier):
    print("\tTotal rows:", dataset.shape[0])
    rows = int(dataset.shape[0] * 0.8)
    train_d = dataset[:rows]
    test_d = dataset[rows:]

    if train_d.shape[0] == 0 or test_d.shape[0] == 0:
        raise Exception(
            "Not enough data",
            dataset.shape[0],
            train_d.shape[0],
            test_d.shape[0],
        )

    # this checks for 0 values
    if train_d.shape[0] == 0 or test_d.shape[0] == 0:
        raise Exception(f"\tNot enough data, train: {train_d.shape[0]}, test: {test_d.shape[0]}")

    print("\tTraining rows:", train_d.shape[0])

    model.fit(train_d[COLUMNS], train_d["Stop"])

    # Valutare il modello utilizzando i dati di test
    prediction = model.predict(test_d[COLUMNS])

    df = pd.DataFrame(
        {
            "Prediction": prediction,
            "Real": test_d["Stop"],
            "Correct": prediction == test_d["Stop"],
        }
    )

    df = df[df["Real"] != "No Stop"]

    res = df["Correct"].value_counts()

    errors = res[False] if False in res else 0

    if df.shape[0] == 0:
        raise Exception("Dataset is not usable")

    percentile = int(errors / df.shape[0] * 100)
    print(f"\tPrediction errors {percentile}%")
    if percentile == 0:
        print("---------")
        print(errors, df.shape[0])

        print(df)
        print(res)

    return percentile

model = HistGradientBoostingClassifier()

average = 0
usables = 0
# for machine_id in train_data["MachineId"].unique():
#     print("Machine", machine_id)

#     dataset = train_data[train_data["MachineId"] == machine_id]

#     try:
#         average += calculateModel(dataset, model)
#         usables += 1
#     except Exception as e:
#         print("\t", e)
print(f"percentile: {calculateModel(train_data, model)}%")

if usables == 0:
    print("No usable data")
else:
    average = average / usables
    print(f"Avg Error: {average:.2f}%")

	Total rows: 1284491
	Training rows: 1027592
	Prediction errors 50%
percentile: 50%
No usable data
